In [49]:
import requests
import numpy as np
import pandas as pd
import pandas_profiling
import json
import os
import matplotlib.pyplot as plt

pd.options.display.max_columns = None

In [50]:
os.chdir(r'C:\Users\2bogu\Desktop\Sringboard_Materials\capstone2\data\external')

df = pd.read_csv('Green_Jobs_-_Green_New_York__GJGNY__Residential_Loan_Portfolio___Beginning_November_2010.csv')

In [51]:
df.describe()

,ORIGINAL LOAN AMOUNT,ORIGINAL TERM,INTEREST RATE,CREDIT SCORE CURRENT HIGH,CREDIT SCORE,CREDIT SCORE NAME 2,DEBT TO INCOME,LAST PAYMENT AMOUNT,PAYMENT AMOUNT,CURRENT BALANCE,UNAPPLIED CASH,DELINQUENT AMOUNT,TOTAL INTEREST PAID,TOTAL LATE CHARGE PAID,PAYMENTS REMAINING,DAYS DELINQUENT,INSTALLATION ZIP,Months Since Origination,Payments Made
count,27385.000000,27385.000000,27385.000000,26930.000000,26863.000000,12149.000000,26938.000000,27385.000000,27385.000000,27385.000000,27385.00000,27385.000000,27385.000000,27385.000000,27385.000000,27385.000000,27385.000000,27385.000000,27385.000000
mean,11651.190126,168.875991,3.872746,741.474341,734.583814,739.132604,0.335567,693.793835,91.993460,7479.760636,2.95803,80.927554,1312.280245,4.986671,102.299142,6.666898,12734.122622,51.724495,66.574344
std,6688.314789,30.736999,1.049811,53.634128,55.735131,54.785011,2.678848,2573.558308,53.368043,6245.036301,118.81430,570.485937,1031.489469,22.687938,58.677823,191.766005,1311.028596,25.555436,51.117475
min,61.000000,1.000000,2.990000,512.000000,479.000000,472.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-18972.000000,-262.000000,0.000000,0.000000,-3375.000000,1443.000000,0.000000,0.000000
25%,6095.000000,180.000000,3.490000,707.000000,698.000000,705.000000,0.190000,53.000000,49.000000,2290.000000,0.00000,0.000000,465.000000,0.000000,71.000000,-27.000000,11738.000000,36.000000,35.300000
50%,10473.000000,180.000000,3.490000,755.000000,747.000000,753.000000,0.310000,93.000000,83.000000,6408.000000,0.00000,0.000000,1078.000000,0.000000,124.000000,-15.000000,12518.000000,51.000000,53.000000
75%,16167.000000,180.000000,3.490000,785.000000,781.000000,784.000000,0.430000,158.000000,127.000000,11646.000000,0.00000,0.000000,1972.000000,0.000000,142.000000,-4.000000,14108.000000,66.000000,80.000000
max,25000.000000,180.000000,8.490000,837.000000,837.000000,833.000000,437.500000,25525.000000,455.000000,25000.000000,19454.00000,15558.000000,6317.000000,1081.000000,180.000000,3277.000000,14905.000000,114.000000,180.000000


In [52]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27385 entries, 0 to 27384
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Reporting Period           27385 non-null  object 
 1   CONTRACT DATE              27385 non-null  object 
 2   SETUP DATE                 27385 non-null  object 
 3   ORIGINAL LOAN AMOUNT       27385 non-null  float64
 4   ORIGINAL TERM              27385 non-null  int64  
 5   ORIGINAL MATURITY DATE     27385 non-null  object 
 6   CURRENT MATURITY DATE      27385 non-null  object 
 7   INTEREST RATE              27385 non-null  float64
 8   CREDIT SCORE CURRENT HIGH  26930 non-null  float64
 9   CREDIT SCORE               26863 non-null  float64
 10  CREDIT SCORE NAME 2        12149 non-null  float64
 11  DEBT TO INCOME             26938 non-null  float64
 12  FIRST PAYMENT DATE         27385 non-null  object 
 13  LAST PAYMENT DATE          27064 non-null  obj

In [53]:
df['ACCOUNT CODE'].unique()

array(['Paid in Full', 'SurePay/Automatic Debit Payment',
       'Check/Money Order Payment (with coupon)', 'Legal Category',
       'Soft Hold', 'On Hold', 'Notice of Bankruptcy (written)',
       'Canceled', 'Notice of Bankruptcy (verbal)', 'Management Hold',
       'SurePay/Automatic Debit Payment (on hold)',
       'Collections Terminated (by customer)', 'Dispute of Debt'],
      dtype=object)

In [54]:
# GETTING ONLY COMPLETED LOANS, EITHER PAYED, CANCELED OR BANKRUPT
df = df.loc[(df['ACCOUNT CODE'] == 'Paid in Full')
      |(df['ACCOUNT CODE'] == 'Canceled')
      |(df['ACCOUNT CODE'] == 'Collections Terminated (by customer)')
      |(df['ACCOUNT CODE'] == 'Notice of Bankruptcy (written)')
      |(df['ACCOUNT CODE'] == 'Notice of Bankruptcy (verbal)')]

In [55]:
df['bad'] = df['ACCOUNT CODE'].apply(lambda x: 0 if x == 'Paid in Full' else 1)

In [56]:
df.bad.value_counts()

0    4903
1     352
Name: bad, dtype: int64

In [57]:
df = df.drop(['LAST PAYMENT AMOUNT', 'LAST PAYMENT DATE', 'FIRST PAYMENT DATE', 
              'DELINQUENT AMOUNT', 'DAYS DELINQUENT', 'Reporting Period', 
              'CREDIT SCORE CURRENT HIGH', 'NEXT PAYMENT DUE DATE', 'PAYMENTS REMAINING', 
              'PROPERTY COUNTY', 'UTILITY', 'INSTALLATION CITY', 'INSTALLATION ZIP', 
              'Contractor Name', 'Georeference', 'UNAPPLIED CASH', 'TOTAL LATE CHARGE PAID', 
              'Months Since Origination', 'CURRENT BALANCE'], axis=1)
df['co-signed'] = np.where(pd.notnull(df['CREDIT SCORE NAME 2']), 1, 0)
df['CREDIT SCORE NAME 2'] = df['CREDIT SCORE NAME 2'].fillna(df['CREDIT SCORE'])
df['SUCCESSOR NUMBER'].fillna('NONE', inplace = True)
df['CANCEL REASON'].fillna('NONE', inplace = True)
df['TYPE OF BANKRUPTCY'].fillna('NONE', inplace = True)
df['avg_credit_score'] = df[['CREDIT SCORE', 'CREDIT SCORE NAME 2']].mean(axis=1)
df.drop(['CREDIT SCORE', 'CREDIT SCORE NAME 2'], inplace=True, axis=1)

In [58]:
# GROUPS MINORITY PURPOSES INTO SINGLE GROUPE
df.loc[(df['Purpose'] == 'Renewable Heat NY (RHNY)') 
       | (df['Purpose'] == 'Renewable Heat NY (RHNY)') 
       | (df['Purpose'] == 'Air Source Heat Pump (ASHP)') 
       | (df['Purpose'] == 'Ground Source Heat Pump (GSHP)')
       | (df['Purpose'] == 'Solar Thermal'), 'Purpose'] = 'Other'  

In [59]:
# CREATES SUCCESSOR NUMBER FLAG
df['has_successor_num'] = df['SUCCESSOR NUMBER'].apply(lambda x: 0 if x == 'NONE' else 1)

# DROPS ORIGINAL COLUMN
df.drop('SUCCESSOR NUMBER', axis=1, inplace=True)

In [60]:
df.Pledged.value_counts()

Series 2015A EE Bonds         1378
Series 2013A EE Bonds         1288
Unpledged                     1081
Series 2015A PV Bonds          771
Series 2016A EE Bonds          448
Series 2018A PV Bonds          154
Series 2019A EE & PV Bonds     135
Name: Pledged, dtype: int64

In [61]:
# CREATES BINARY COLUMN FOR PLEDGED VS UNPLEDGED LOANS
df['unpledged'] = df['Pledged'].apply(lambda x: 1 if x == 'Unpledged' else 0)

# DROPS ORIGINAL PLEDGED COLUMN
df.drop('Pledged', axis=1, inplace=True)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5255 entries, 0 to 27383
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CONTRACT DATE           5255 non-null   object 
 1   SETUP DATE              5255 non-null   object 
 2   ORIGINAL LOAN AMOUNT    5255 non-null   float64
 3   ORIGINAL TERM           5255 non-null   int64  
 4   ORIGINAL MATURITY DATE  5255 non-null   object 
 5   CURRENT MATURITY DATE   5255 non-null   object 
 6   INTEREST RATE           5255 non-null   float64
 7   DEBT TO INCOME          5129 non-null   float64
 8   PAYMENT AMOUNT          5255 non-null   float64
 9   TOTAL INTEREST PAID     5255 non-null   float64
 10  ACCOUNT CODE            5255 non-null   object 
 11  ACCOUNT CODE DATE       5255 non-null   object 
 12  CANCEL REASON           5255 non-null   object 
 13  TYPE OF BANKRUPTCY      5255 non-null   object 
 14  Payments Made           5255 non-null  

In [63]:
df['CANCEL REASON'].value_counts()

NONE                          5095
DEATH                           91
CUSTOMER DEFAULTED              25
SETTLEMENT                      16
CANCEL DUE TO BANKRUPTCY        13
INVOLUNTARY REPOSESSION         12
VOLUNTARY REPOSESSION            1
DEVELOPER RELEASED ACCOUNT       1
VOLUNTARY CANCEL                 1
Name: CANCEL REASON, dtype: int64

In [64]:
df.drop(df[(df['CANCEL REASON'] == 'DEATH') 
           | (df['CANCEL REASON'] == 'DEVELOPER RELEASED ACCOUNT')].index,inplace=True)

In [65]:
df.drop(['ACCOUNT CODE', 'CANCEL REASON', 'TYPE OF BANKRUPTCY', 'TOTAL INTEREST PAID',
         'Payments Made'], axis=1, inplace=True)

In [66]:
# CREATE DATE FEATURES HERE
# CREATE DATE FEATURES HERE
# CREATE DATE FEATURES HERE
# IF NEEDED

df.drop(['CONTRACT DATE', 'SETUP DATE', 'ORIGINAL MATURITY DATE', 'CURRENT MATURITY DATE', 'ACCOUNT CODE DATE'], 
        axis=1, inplace=True)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5163 entries, 0 to 27383
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ORIGINAL LOAN AMOUNT  5163 non-null   float64
 1   ORIGINAL TERM         5163 non-null   int64  
 2   INTEREST RATE         5163 non-null   float64
 3   DEBT TO INCOME        5038 non-null   float64
 4   PAYMENT AMOUNT        5163 non-null   float64
 5   Purpose               5163 non-null   object 
 6   Loan Type             5163 non-null   object 
 7   Underwriting          5163 non-null   object 
 8   bad                   5163 non-null   int64  
 9   co-signed             5163 non-null   int32  
 10  avg_credit_score      5034 non-null   float64
 11  has_successor_num     5163 non-null   int64  
 12  unpledged             5163 non-null   int64  
dtypes: float64(5), int32(1), int64(4), object(3)
memory usage: 544.5+ KB


In [68]:
df[df['DEBT TO INCOME'].isna()]

,ORIGINAL LOAN AMOUNT,ORIGINAL TERM,INTEREST RATE,DEBT TO INCOME,PAYMENT AMOUNT,Purpose,Loan Type,Underwriting,bad,co-signed,avg_credit_score,has_successor_num,unpledged
1339,7541.00,165,3.49,NaN,58.00,Energy Efficiency (EE),On Bill Recovery,Tier 1,0,0,NaN,1,0
2366,13257.43,175,3.49,NaN,96.77,Solar (PV),On Bill Recovery,Tier 2,0,0,NaN,1,1
2494,13835.47,157,3.49,NaN,109.89,Energy Efficiency (EE),On Bill Recovery,Tier 2,0,0,NaN,1,1
3696,22761.00,163,3.49,NaN,176.00,Solar (PV),On Bill Recovery,Tier 1,0,0,NaN,1,0
3740,22977.00,165,3.49,NaN,176.00,Solar (PV),On Bill Recovery,Tier 1,0,0,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27092,19634.00,142,3.49,NaN,169.00,Energy Efficiency (EE),On Bill Recovery,Tier 1,1,0,NaN,1,0
27275,3085.00,171,3.49,NaN,23.00,Energy Efficiency (EE),On Bill Recovery,Tier 1,0,0,NaN,1,0
27284,6062.00,125,3.49,NaN,58.00,Energy Efficiency (EE),On Bill Recovery,Tier 2,0,0,NaN,1,1
27314,5527.00,158,3.49,NaN,44.00,Solar (PV),On Bill Recovery,Tier 1,0,0,NaN,1,0


In [69]:
# FILLING DEBT TO INCOME AND CREDIT SCORE WITH THE AVERAGE

di_avg = df['DEBT TO INCOME'].mean()
df['DEBT TO INCOME'].fillna(round(di_avg, 2), inplace =True)

cs_avg = df['avg_credit_score'].mean()
df['avg_credit_score'].fillna(round(cs_avg, 1), inplace =True)

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5163 entries, 0 to 27383
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ORIGINAL LOAN AMOUNT  5163 non-null   float64
 1   ORIGINAL TERM         5163 non-null   int64  
 2   INTEREST RATE         5163 non-null   float64
 3   DEBT TO INCOME        5163 non-null   float64
 4   PAYMENT AMOUNT        5163 non-null   float64
 5   Purpose               5163 non-null   object 
 6   Loan Type             5163 non-null   object 
 7   Underwriting          5163 non-null   object 
 8   bad                   5163 non-null   int64  
 9   co-signed             5163 non-null   int32  
 10  avg_credit_score      5163 non-null   float64
 11  has_successor_num     5163 non-null   int64  
 12  unpledged             5163 non-null   int64  
dtypes: float64(5), int32(1), int64(4), object(3)
memory usage: 544.5+ KB


In [71]:
df.Underwriting.value_counts()

Tier 1    4548
Tier 2     615
Name: Underwriting, dtype: int64

In [72]:
# CREATES FLAG FOR UNDERWRITTING T1, ONLY T1 AND T2 IN SET
df['underwritten_t1'] = df['Underwriting'].apply(lambda x: 1 if x == 'Tier 1' else 0)

df.drop('Underwriting', axis=1, inplace=True)

In [73]:
df['underwritten_t1'].value_counts()

1    4548
0     615
Name: underwritten_t1, dtype: int64

In [74]:
df['Loan Type'].value_counts()

Smart Energy        3477
On Bill Recovery    1686
Name: Loan Type, dtype: int64

In [75]:
# CREATES BOOL FOR LOAN TYPE, 0 = ON BILL RECOVERY
df['loan_type_smart_energy'] = df['Loan Type'].apply(lambda x: 1 if x == 'Smart Energy' else 0)

df.drop('Loan Type', axis= 1, inplace=True)

In [76]:
# CREATES BOOL FOR PURPOSE, DOUBLE ZERO IS OTHER
df['purpose_ee'] = df['Purpose'].apply(lambda x: 1 if x == 'Energy Efficiency (EE)' else 0)

df['purpose_sol'] = df['Purpose'].apply(lambda x: 1 if x == 'Solar (PV)' else 0)

df.drop('Purpose', axis=1, inplace=True)

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5163 entries, 0 to 27383
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ORIGINAL LOAN AMOUNT    5163 non-null   float64
 1   ORIGINAL TERM           5163 non-null   int64  
 2   INTEREST RATE           5163 non-null   float64
 3   DEBT TO INCOME          5163 non-null   float64
 4   PAYMENT AMOUNT          5163 non-null   float64
 5   bad                     5163 non-null   int64  
 6   co-signed               5163 non-null   int32  
 7   avg_credit_score        5163 non-null   float64
 8   has_successor_num       5163 non-null   int64  
 9   unpledged               5163 non-null   int64  
 10  underwritten_t1         5163 non-null   int64  
 11  loan_type_smart_energy  5163 non-null   int64  
 12  purpose_ee              5163 non-null   int64  
 13  purpose_sol             5163 non-null   int64  
dtypes: float64(5), int32(1), int64(8)
memor

In [78]:
df.to_csv(r'C:\Users\2bogu\Desktop\Sringboard_Materials\capstone2\data\interim\new_new', index = False)